In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import platform

import os, warnings # 경고메세지 라이브러리
import re   # 정규표현식
import time
warnings.filterwarnings(action='ignore') # 경고메세지 무시

import pandas as pd

In [2]:
# OS 별로 웹드라이버 로드(windows, mac)
if platform.system() == "Windows":
    driver = webdriver.Chrome('../chromedriver_90')
elif platform.system()=="Darwin":
    driver = webdriver.Chrome('../chromedriver')
else:
    print("Unknown System")
    
url = 'http://www.lotteria.com/Shop/Shop_List.asp#devCallShopList'
driver.get(url)
time.sleep(3)

In [3]:
# 마지막 페이지 이동
end_btn = driver.find_element_by_css_selector('a.go.end')
end_btn.click()
time.sleep(5)

In [4]:
# 마지막 페이지 확인
end_num = 0

while 1 : 
    now_page = int(driver.find_element_by_css_selector('span>strong').text)
    now_page

    if now_page > 10 :
        print('마지막 페이지 확인')
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')

        end_page = soup.select_one('a.go.end')

        end_num = int(end_page.find_all_previous()[2].text)
        end_num
        break
    else :
        time.sleep(2)


마지막 페이지 확인


In [5]:
idx_list = []
code_list = []

# 1부터 마지막 페이지까지 반복
for page_num in range (1,end_num + 1):

    # 자바스크립트 이용하여 페이지 이동
    driver.execute_script(f"goPage({page_num})".format())
    
    # 현재 페이지 상태 확인 후 페이지 이동이 될 때까지 2초씩 대기 
    while 1 : 
        now_page = int(driver.find_element_by_css_selector('span>strong').text)
        now_page
        
        if now_page == page_num :
            print(f'{page_num} 확인', end='')
            break
        else :
            time.sleep(2)

    # 현재 페이지 소스 객체 생성
    page_one = driver.page_source
    soup = BeautifulSoup(page_one, 'lxml')        
    place_num = soup.find_all('td',class_='first num')

    # url 주소 안의 idx와 StoreCode 추출
    for one in place_num:
        first_one = one.a.get("onclick")[7:19][0:4]    
        second_one = one.a.get("onclick")[7:19][-5:]

        idx_list.append(first_one)
        code_list.append(second_one)
        
    print(f', {page_num} 완료')

1 확인, 1 완료
2 확인, 2 완료
3 확인, 3 완료
4 확인, 4 완료
5 확인, 5 완료
6 확인, 6 완료
7 확인, 7 완료
8 확인, 8 완료
9 확인, 9 완료
10 확인, 10 완료
11 확인, 11 완료
12 확인, 12 완료
13 확인, 13 완료
14 확인, 14 완료
15 확인, 15 완료
16 확인, 16 완료
17 확인, 17 완료
18 확인, 18 완료
19 확인, 19 완료
20 확인, 20 완료
21 확인, 21 완료
22 확인, 22 완료
23 확인, 23 완료
24 확인, 24 완료
25 확인, 25 완료
26 확인, 26 완료
27 확인, 27 완료
28 확인, 28 완료
29 확인, 29 완료
30 확인, 30 완료
31 확인, 31 완료
32 확인, 32 완료
33 확인, 33 완료
34 확인, 34 완료
35 확인, 35 완료
36 확인, 36 완료
37 확인, 37 완료
38 확인, 38 완료
39 확인, 39 완료
40 확인, 40 완료
41 확인, 41 완료
42 확인, 42 완료
43 확인, 43 완료
44 확인, 44 완료
45 확인, 45 완료
46 확인, 46 완료
47 확인, 47 완료
48 확인, 48 완료
49 확인, 49 완료
50 확인, 50 완료
51 확인, 51 완료
52 확인, 52 완료
53 확인, 53 완료
54 확인, 54 완료
55 확인, 55 완료
56 확인, 56 완료
57 확인, 57 완료
58 확인, 58 완료
59 확인, 59 완료
60 확인, 60 완료
61 확인, 61 완료
62 확인, 62 완료
63 확인, 63 완료
64 확인, 64 완료
65 확인, 65 완료
66 확인, 66 완료
67 확인, 67 완료
68 확인, 68 완료
69 확인, 69 완료
70 확인, 70 완료
71 확인, 71 완료
72 확인, 72 완료
73 확인, 73 완료
74 확인, 74 완료
75 확인, 75 완료
76 확인, 76 완료
77 확인, 77 완료
78 확인, 78 완료
79 확

In [6]:
# 웹드라이버 종료
driver.close()

In [7]:
# idx_list의 길이 확인
len(idx_list)

1306

In [8]:
store_names = [] # 지점명
store_addrs = [] # 지점주소

# user-agent 정보를 넣어서 웹브라우저 정보를 변경
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36'}
    
for i in range(len(idx_list)) :
    
    # 메인 화면에 가게 이름의 url을 통해 store_page(html)정보를 담는다.
    url = f'http://www.lotteria.com/Shop/Shop_View.asp?Idx={idx_list[i]}&StoreCode={code_list[i]}'
    store_page = requests.get(url, headers=headers).content
    detail = BeautifulSoup(store_page, 'lxml')

    store_name = detail.find("h3").text
    store_names.append(store_name)
    store_addr = detail.find("td",class_='rt').span.text.strip()
    store_addrs.append(store_addr)
    
#     time.sleep(1)
    
    print(f'{i}, {store_name} : {store_addr}')

0, 가평휴게소(상) : 경기 가평군 설악면 미사리로 544
1, 아산장재 : 서울 용산구 한강대로71길 47
2, 수원파장 : 경기 수원시 장안구 파장로 91
3, 롯데더몰여수 : 전남 여수시 국포1로 36
4, 김포마산 : 경기 김포시 김포한강8로148번길 5
5, 신정네거리역 : 서울 양천구 신월로 297
6, 대구성당D/T : 대구 달서구 구마로 209
7, 진주혁신 : 경남 진주시 소호로 12
8, 영암대불 : 전남 영암군 삼호읍 대불주거로 145
9, 안산사동 : 경기 안산시 상록구 항가울로 28
10, 의정부고산 : 경기 의정부시 문충로 116
11, 용인휴게소(하) : 경기 용인시 처인구 양지면 주북로94번길 30-1
12, 진영휴게소(하) : 경남 김해시 진영읍 하계로94번길 94-4
13, 포항에비뉴 : 경북 포항시 남구 중흥로 77
14, 송도달빛공원 : 인천 연수구 송도동 29-1번지(아트센터대로 169번길 100)
15, 광명역사 : 경기 광명시 광명역로 광명역 동편 맞이방
16, 문경시청 : 경북 문경시 시청1길 5
17, 청계산입구역 : 서울 서초구 청계산로 201
18, 대전목원대 : 대전 서구 도안북로93번길 7
19, 원주봉산 : 강원 원주시 행구로 69
20, 무안 : 전남 무안군 무안읍 면성1길 129
21, 안산그랑시티자이 : 경기 안산시 상록구 해양1로 199, 그랑시티 PORT AVENUE
22, 시화거북섬 : 경기 시흥시 거북섬5길 4
23, 인천왕길역 : 인천 서구 봉화로 49
24, 홈플러스강동 : 굽은다리역 앞 홈플러스강동점 1층
25, 서천장항 : 충남 서천군 장항읍 장서로 73
26, 경기북부청사 : 경기 의정부시 장곡로 620
27, 송도랜드마크시티 : 인천 연수구 랜드마크로 68
28, 청주동남 : 충북 청주시 상당구 중고개로 141번길 13-16
29, 남양주덕소 : 경기 남양주시 와부읍 수레로 100
30, 동탄산척 : 경기 화성시 동탄순환대로 245
31, 감일 : 감일백제로 109 1층
32, 대구

262, 정읍상동 : 전북 정읍시 신경2길 28
263, 영등포롯데 : 서울 영등포구 경인로 846 롯데백화점 영등포점 지하1층
264, 영등포역사 : 서울 영등포구 경인로102길 13
265, 대구가창D/T : 대구 달성군 가창면 가창로 1075
266, 포일IT : 경기 의왕시 포일세거리로 10
267, 안성맞춤휴게소(상) : 경기 안성시 서운면 평택제천고속도로 41
268, 안성맞춤휴게소(하) : 경기 안성시 서운면 평택제천고속도로 40
269, 죽암휴게소(하)D/T : 충북 청주시 서원구 현도면 경부고속도로 289
270, 용인모현 : 경기 용인시 처인구 모현면 백옥대로 2387
271, 부산해운대좌동 : 부산 해운대구 좌동순환로 74
272, 평택안중D/I : 경기 평택시 안중읍 안현로서9길 5
273, 화성반월 : 경기 화성시 영통로27번길 1
274, 남성역 : 서울 동작구 사당로 200
275, 울산옥동타워 : 울산 남구 문수로 399
276, 영주가흥 : 경북 영주시 대동로31번길 2
277, 전주하가D/I : 전북 전주시 덕진구 덕진동2가 392-7
278, 용인천리 : 경기 용인시 처인구 이동면 백옥대로 641
279, 부산재송 : 부산 해운대구 재반로 86
280, 고성간성 : 강원 고성군 간성읍 간성로 59-1
281, 오창2산단 : 충북 청주시 청원구 오창읍 주성1길 5
282, 부산민락 : 부산 수영구 광안해변로 282
283, 이천하이닉스 : 경기 이천시 경충대로 2096-4
284, 파주당동 : 경기 파주시 문산읍 방촌로 1719-57
285, 현풍테크노폴리스 : 대구 달성군 유가면 테크노대로 61
286, 수지풍덕천 : 경기 용인시 수지구 풍덕천로 133
287, 의정부민락 : 경기 의정부시 천보로 44, 해동본타워 1층 102호
288, 광주백운 : 광주 남구 독립로 100-1
289, 세종종촌 : 세종  도움1로 108
290, 부산신호 : 부산 강서구 신호산단2로27번길 3
291, 전남함평 : 전남 함평군 함평읍 중앙길 

516, 장흥 : 전남 장흥군 장흥읍 중앙로54-1
517, 문정로데오 : 서울 송파구 동남로 134
518, 익산영등 : 전북 익산시 영등동 약촌로 243
519, 안양호계 : 경기 안양시 동안구 경수대로 561
520, 전주중화산 : 전북 전주시 완산구 백제대로 224
521, 대전가수원 : 대전 서구 계백로 1161
522, 군산조촌D/T : 전북 군산시 조촌로 111
523, 김해아울렛식당가 : 경남 김해시 장유로 469
524, 역삼 : 서울 강남구 테헤란로 116
525, 대구대곡역 : 대구 달성군 화원읍 비슬로 2723
526, 완주봉동 : 전북 완주군 봉동읍 완주산단9로15
527, 잠실새내 : 서울 송파구 올림픽로 114
528, 부산하단 : 부산 사하구 낙동대로 457
529, 천호역 : 서울 강동구 올림픽로 654
530, 안양중앙 : 경기도 안양시 만안구 안양4동 710-57
531, 홈플러스경산 : 경북 경산시 경안로 288
532, 별내신도시 : 경기 남양주시 별내동 순화궁로 347
533, 홈플러스오산세교 : 경기 오산시 청학로 238
534, 대전가양D/T : 대전 대덕구 동서대로 1772(비래동)
535, 대전복수 : 대전 서구 복수동로 62번지 1층
536, 수원역사 : 경기 수원시 팔달구 덕영대로 924
537, 안산반월 : 경기 안산시 상록구 건건2길 6
538, 창동역 : 서울 도봉구 노해로 63길78
539, 송산휴게소 : 경기 화성시 송산면 고정리 349-26
540, 인천동양 : 인천 계양구 동양로111
541, 홈플러스합정 : 서울 마포구 양화로 45
542, 광주운암DI : 광주 북구 북문대로 177(운암동)
543, 여주375아울렛 : 경기 여주시 명품1로 22-5
544, 의정부녹양역 : 경기 의정부시 체육로298-33 CS타워 1,2층
545, 용인보라 : 경기 용인시 기흥구 한보라2로 5
546, 별내 : 경기 남양주시 별내동 별내 5로 5번길 11
547, 지행역 : 경기 동두천시 지행로 60
548, 홈

774, 부산정관 : 부산 기장군 정관면 정관로 560
775, 사가정역 : 서울 중랑구 면목로 333
776, 광주동림 : 광주 광역시 서구 동천동626
777, 용인둔전 : 경기 용인시 포곡읍 포곡로 107번길 2
778, 인천간석 : 인천 남구 주안로219
779, 일산후곡 : 경기 고양시 일산서구 일산로 544
780, 인천월미마이랜드 : 인천 중구 월미문화로 81
781, 성남단대 : 경기 성남시 중원구 산성대로 372번길 15
782, 고덕역 : 서울 강동구 동남로73길 26 02호
783, 목포 : 전남 목포시 영산로 59번길 30
784, 의정부호원 : 경기 의정부시 평화로238
785, 목포북항 : 전남 목포시 청호로 173
786, 미아삼거리 : 서울 성북구 도봉로 29
787, 인천남동공단 : 인천 남동구 논현로 40 1층
788, 롯데마트광주수완 : 광주 광산구 장신로 98
789, 부천뉴코아 : 경기 부천시 원미구 송내대로 239(상동)
790, 영등포타임스퀘어B2 : 서울 영등포구 영중로 15(영등포동 4가)B2층
791, 김해율하 : 경남 김해시 장유동 율하3로 42
792, 대구서부정류장 : 대구 달서구 구마로 260
793, 신도림역 : 서울 구로구 경인로661(신도림동)
794, 대구서재 : 대구 달성군 다사읍 서재본길 4-1
795, 롯데마트서산 : 충남 서산시 충의로 27 롯데마트 2층
796, 진주금산 : 경남 진주시 금산면 금산순환로 4
797, 알펜시아리조트 : 강원 평창군 대관령면 솔봉로 325
798, 김포장기 : 경기 김포시 김포한강 1로 78번길 5(장기동)
799, 광주학동 : 광주 동구 남문로 682 (학동)
800, 전주동산 : 전북 전주시 덕진구 편운로 6
801, 평촌귀인 : 경기 안양시 동안구 평촌대로 139
802, 고양행신역 : 경기 고양시 덕양구 행신동 무원로6번길 50
803, 인천올리브 : 인천 남동구 남동대로 880
804, 롯데마트고양 : 경기 고양시 덕양구 행신동 중앙동 558번길 21


1032, 홈플러스센텀시티 : 부산 해운대구 센텀동로 1-224
1033, 부산안락 : 부산 동래구 안락로 134
1034, 남양주청학 : 경기 남양주시 청학로중앙길15
1035, 동암남부 : 동암남로 5
1036, 포항흥해 : 경북 포항시 북구 흥해읍 흥해로 38
1037, 창원도계 : 경남 창원시 의창구 도계로 99
1038, 일산탄현 : 경기 고양시 일산서구 일현로 47
1039, 홈플러스동수원 : 경기 수원시 팔달구 호원로 257
1040, 광양 : 전남 광양시 광양읍 매천로835
1041, 명지대학 : 서울시 서대문구 남가좌2동 324-31
1042, 개봉역 : 서울 구로 경인로 40길 47 1F
1043, 롯데마트울산 : 울산 남구 달동 삼산로 74(달동)
1044, 양산덕계 : 경남 양산시 덕계로 78
1045, 삼천포벌리 : 경남 사천시 주공로 9
1046, 광주하남 : 광주광역시 광산구 월곡1동 538-2
1047, 인천강화 : 인천 강화군 강화읍 강화대로 419
1048, 광주진월 : 광주 남구 서문대로 662
1049, 궁전라벤더 : 대구 동구 아양로 52
1050, 익산부송 : 전북 익산시 부송로66
1051, 전주아중 : 전북 전주시 덕진구 무삼지로 66
1052, 울진D/I : 경북 울진군 울진읍 읍내리 47
1053, 대전중촌 : 대전 중구 목중로 22 1층 롯데리
1054, 장현 : 경기 남양주시 진접읍 장현로 93
1055, 인천용현 : 인천 남구 낙섬중로 42
1056, 금촌로터리 : 경기 파주시 황골로 39
1057, 남부터미널 : 서울 서초구 효령로 289 장곡빌딩 103호
1058, 하남신장 : 경기 하남시 대청로73
1059, 홈플러스울산 : 울산 중구 번영로 475
1060, 킴스순천 : 전남 순천시 비봉2길 4-12
1061, 화성남양 : 화성시 남양동 남양사거리 안쪽 아이리스프라자 1층
1062, 안산본오 : 경기 안산시 상록구 샘골로 150 동성빌딩 101
1063, 인천작전 : 인천 계양구 안남로 548
1064

1298, 동서울터미널 : 서울 광진구 구의3동546-1
1299, 봉천 : 서울 관악구 관악로212
1300, 강릉 : 강원 강릉시 경강로 2106
1301, 원주일산 : 강원 원주시 천사로 213
1302, 성남 : 경기 성남시 수정구 수정로 181
1303, 대전중앙 : 대전광역시 중구 선화동 3-3번지
1304, 홈서비스과천 : 경기 과천시 별양동 19-4
1305, 홈서비스부암(부산역) : 부산 부산진구 부암동 722-3번지


In [9]:
# URLError: <urlopen error [Errno 11001] getaddrinfo failed> 

In [10]:
# store_names, store_addrs 길이 확인
len(store_names), len(store_addrs)

(1306, 1306)

In [11]:
# 수집된 정보로 DataFrame 생성
dat = {'지점명':store_names, '주소':store_addrs}
dat = pd.DataFrame(dat)
dat.head()

,지점명,주소
0,가평휴게소(상),경기 가평군 설악면 미사리로 544
1,아산장재,서울 용산구 한강대로71길 47
2,수원파장,경기 수원시 장안구 파장로 91
3,롯데더몰여수,전남 여수시 국포1로 36
4,김포마산,경기 김포시 김포한강8로148번길 5


In [12]:
# 수집된 데이터 전처리 전 저장
dat.to_excel('lotteria_all_store_v1.xlsx', index=False)
dat.to_csv('lotteria_all_store_v1.csv', index=False)
dat.head()

,지점명,주소
0,가평휴게소(상),경기 가평군 설악면 미사리로 544
1,아산장재,서울 용산구 한강대로71길 47
2,수원파장,경기 수원시 장안구 파장로 91
3,롯데더몰여수,전남 여수시 국포1로 36
4,김포마산,경기 김포시 김포한강8로148번길 5


In [13]:
# 수집된 데이터 가져오기
dat = pd.read_excel('./lotteria_all_store_v1.xlsx')
dat

,지점명,주소
0,가평휴게소(상),경기 가평군 설악면 미사리로 544
1,아산장재,서울 용산구 한강대로71길 47
2,수원파장,경기 수원시 장안구 파장로 91
3,롯데더몰여수,전남 여수시 국포1로 36
4,김포마산,경기 김포시 김포한강8로148번길 5
5,신정네거리역,서울 양천구 신월로 297
6,대구성당D/T,대구 달서구 구마로 209
7,진주혁신,경남 진주시 소호로 12
8,영암대불,전남 영암군 삼호읍 대불주거로 145
9,안산사동,경기 안산시 상록구 항가울로 28


In [14]:
# 주소를 행정구역으로 나눔
dat[['주소1', '주소2', '주소3']] = pd.DataFrame(dat['주소'].str.split(' ', 2).tolist())
dat.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기 가평군 설악면 미사리로 544,경기,가평군,설악면 미사리로 544
1,아산장재,서울 용산구 한강대로71길 47,서울,용산구,한강대로71길 47
2,수원파장,경기 수원시 장안구 파장로 91,경기,수원시,장안구 파장로 91
3,롯데더몰여수,전남 여수시 국포1로 36,전남,여수시,국포1로 36
4,김포마산,경기 김포시 김포한강8로148번길 5,경기,김포시,김포한강8로148번길 5


In [15]:
# 주소1에 이상치를 확인
dat['주소1'].unique()

array(['경기', '서울', '전남', '대구', '경남', '경북', '인천', '대전', '강원', '굽은다리역',
       '충남', '충북', '감일백제로', '세종', '광주', '부산', '제주', '울산', '전북', '경기도',
       '충청북도', '경상남도', '전라북도', '진안군', '인천공항', '동대문구', '부산광역시', '경남남도',
       '문경새재도립공원', '번동사거리', '서울특별시', '대전시', '롯데마트', '동해시', '중곡동', '대전광역시',
       '지하철', '강남역', '방학', '광혜원시외버스정류장', '성수역1번출구', '서울시', '도화', '용인시',
       '홈배달', '수완지구', '별곡리', '부산시', '잠실', '백마역', '2호선', '청량리역3번출구',
       '충청남도', '회룡역', '진건초등학교', '두정역에서', '죽곡', '주례역', '야탑역4번출구', '창원시',
       '대우자동차', '동암남로', '광주광역시', '화성시', '단계동', '까치산역3번출구에서', '천안시',
       '모래내시장', '남항동', '순천연향동', '논산', '부천시', '금촌역', '강원도', '제천시'],
      dtype=object)

In [16]:
# 주소1의 줄임말을 풀어서 치환
addr = {'서울시':'서울특별시',
        '서울':'서울특별시',
        '부산시':'부산광역시',
        '부산':'부산광역시',
        '대구':'대구광역시',
        '인천':'인천광역시',
        '광주':'광주광역시',
        '대전시':'대전광역시',
        '대전':'대전광역시',
        '울산':'울산광역시',
        '세종':'세종특별자치시',
        '경기':'경기도',
        '강원':'강원도',
        '충북':'충청북도',
        '충남':'충청남도',
        '전북':'전라북도',
        '전남':'전라남도',
        '경북':'경상북도',
        '경남':'경상남도',
        '제주':'제주특별자치도'}

dat['주소1'] = dat['주소1'].replace(addr)

dat['주소1'].unique()

array(['경기도', '서울특별시', '전라남도', '대구광역시', '경상남도', '경상북도', '인천광역시', '대전광역시',
       '강원도', '굽은다리역', '충청남도', '충청북도', '감일백제로', '세종특별자치시', '광주광역시',
       '부산광역시', '제주특별자치도', '울산광역시', '전라북도', '진안군', '인천공항', '동대문구', '경남남도',
       '문경새재도립공원', '번동사거리', '롯데마트', '동해시', '중곡동', '지하철', '강남역', '방학',
       '광혜원시외버스정류장', '성수역1번출구', '도화', '용인시', '홈배달', '수완지구', '별곡리', '잠실',
       '백마역', '2호선', '청량리역3번출구', '회룡역', '진건초등학교', '두정역에서', '죽곡', '주례역',
       '야탑역4번출구', '창원시', '대우자동차', '동암남로', '화성시', '단계동', '까치산역3번출구에서',
       '천안시', '모래내시장', '남항동', '순천연향동', '논산', '부천시', '금촌역', '제천시'],
      dtype=object)

In [17]:

# 210702 롯데리아 강남역 이슈
# 네이버지도에 롯데리아 강남역 검색결과 없어서 주소 확인
# 주소 : 강남역 9번이나 10번출구 뉴욕제과 뒷건물
# 뉴욕제과 확인 :         로드뷰 확인결과 2012년 03월까지 있었지만 현재는 없음
# 롯데리아 강남역점 확인 : 로드뷰 확인 결과 2017년 5월까지 있었지만 이후 없어지고 KFC 생김

# 210703 롯데리아 주안D/T 이슈
# 네이버지도에 위치가 확인이 안됨
# 네이버 블로그에 있는 지도로 위치 확인
# 로드뷰 확인결과 2019년 05월까지 있었지만 현재는 없음
   
# 주소1의 이상값을 정상값으로 변경하고 없는 지점은 None처리
def change_addr(x) :
    
    s_name_ori = x['지점명']
    
    # D/T 같이 '/'이 포함된 경우 네이버에 제대로 검색되지 않음
    s_name = s_name_ori
    if '/' in s_name :
        s_name = s_name.replace('/', '')
        
    f_name = f'롯데리아 {s_name}'
    
    url = f'https://search.naver.com/search.naver?query={f_name}'
          
    driver.get(url)
    time.sleep(1)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    # 네이버에 '롯데리아 지점명' 검색하여 주소 획득
    try :

        # case1 : 롯데리아 감일
        full_addr = soup.select_one('span.addr').text
        addr1, addr2, addr3 = full_addr.split(' ', 2)
        print(f'{s_name} : {addr1} {addr2} {addr3}')
        x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3

    except Exception as e :

        print(f'{s_name} : 1. {e}')
        full_addr = ''

        try :
            store_list = soup.select('div._3ZU00')
            
            # case2 : 롯데리아 의정부화룡역
            if len(store_list) == 0 :
                full_addr = soup.select('div._1mAZf')[1].text

                full_addr = full_addr[:full_addr.rindex('지번')]
                addr1, addr2, addr3 = full_addr.split(' ', 2)
                print(f'{s_name} : {addr1} {addr2} {addr3}')
                x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3

            # 검색목록이 여러개인 경우, 지점명과 일치하는 주소를 획득
            for idx, i in enumerate(store_list) :
                
                except_text = ['전북', '전남', '경북', '경남']
                
                # try 구문을 사용해도 되지만 이미 이중으로 try 선언이 되어 굳이 String으로 해결
                # 지점명에 도명이 포함된 경우, 네이버 검색 시 도명이 없이 롯데리아 진안점으로 검색됨
                # 도명을 제거하여 네이버 검색결과 중 일치하는 지점의 주소를 입력
                s_name2 = 'None 선언시 에러가 생기므로 String으로 None 선언'
                if s_name[:2] in except_text :
                    s_name2 = s_name[2:]

                # case3 : 롯데리아 전북진안, 롯데리아 강남역
                if s_name in i.a.text or s_name2 in i.a.text :
                
                    try :
                        xpath = f'//*[@id="loc-main-section-root"]/section/div/div[2]/ul/li[{idx+1}]/div[2]/div[2]/div/span/a/span[2]'
                        down = driver.find_element_by_xpath(xpath)
                    except :
                        # case4 : 롯데리아 경남함안
                        xpath = f'//*[@id="loc-main-section-root"]/section/div/div[2]/ul/li[{idx+1}]/div[1]/div[2]/div/span/a/span[2]'
                        down = driver.find_element_by_xpath(xpath)

                    down.click()
                    time.sleep(1)
                    
                    # 주소에 도로명과, 복사를 뺴고 입력
                    full_addr = driver.find_element_by_css_selector('div._2b9ic').text[3:-2]
                    addr1, addr2, addr3 = full_addr.strip().split(' ', 2)
                    print(f'{s_name} : {addr1} {addr2} {addr3}')
                    x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3
                    
                    break
                    
        except Exception as e:

            print(f'{s_name} : 2. {e}')
            
    if len(full_addr) < 1 :
        # drop 처리하려 했으나 작업 편의성을 위해 None 처리
#         print(f'{s_name_ori} : 삭제')
#         dat.drop( dat.loc[ dat['지점명'] == s_name_ori ].index, axis=0, inplace=True)

        print(f'{s_name_ori} : None처리')
        pass
    else :
        return x

# 웹드라이버 실행
driver = webdriver.Chrome('../chromedriver_90')

# 주소1의 이상값을 정상값으로 변경하고 없는 지점은 None처리
dat = dat.apply(lambda x : change_addr(x) if x['주소1'] not in addr.values() else x, axis=1)

# 웹드라이버 종료
driver.close()

홈플러스강동 : 서울 강동구 양재대로 1571
감일 : 경기 하남시 감일백제로 109
전북진안 : 1. 'NoneType' object has no attribute 'text'
전북진안 : 전라북도 진안군 진안읍 진무로 1093 1층 롯데리아
인천공항제2여객터미널1층 : 인천 중구 제2터미널대로 446
장한평역 : 1. 'NoneType' object has no attribute 'text'
장한평역 : 서울특별시 동대문구 천호대로 401
외대역 : 1. 'NoneType' object has no attribute 'text'
외대역 : 서울특별시 동대문구 휘경로 5-1
경남함안 : 1. 'NoneType' object has no attribute 'text'
경남함안 : 경상남도 함안군 가야읍 함마대로 1514-25
문경새재 : 경북 문경시 문경읍 새재로 928
번동DT : 서울 강북구 한천로 963
롯데마트수원천천 : 경기 수원시 장안구 만석로19번길 25-10
동해쇄운 : 강원 동해시 청운1길 52
중곡 : 1. 'NoneType' object has no attribute 'text'
중곡 : 서울특별시 광진구 면목로 136 창원빌딩
부산미남역 : 부산 동래구 아시아드대로 234
강남역 : 1. 'NoneType' object has no attribute 'text'
강남역 : None처리
방학 : 1. 'NoneType' object has no attribute 'text'
방학 : 서울특별시 도봉구 방학로 173
광혜원 : 충북 진천군 광혜원면 장기길 104
성수역 : 1. 'NoneType' object has no attribute 'text'
성수역 : 서울특별시 성동구 아차산로7길 28 성수쇼핑센타
주안DT : 1. 'NoneType' object has no attribute 'text'
주안DT : 2. list index out of range
주안D/T : None처리
용인죽전1동 : 1. 'NoneType' object h

In [18]:
# null 확인
dat.isnull().sum()

지점명    2
주소     2
주소1    2
주소2    2
주소3    2
dtype: int64

In [19]:
# 주소1의 줄임말을 풀어서 치환하고 None처리 된 행을 제거
dat['주소1'] = dat['주소1'].replace(addr)
dat.dropna(inplace=True)
dat.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전남 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5


In [20]:
# null 확인
dat.isnull().sum()

지점명    0
주소     0
주소1    0
주소2    0
주소3    0
dtype: int64

In [21]:
# 주소1 이상치 확인
dat['주소1'].unique()

array(['경기도', '서울특별시', '전라남도', '대구광역시', '경상남도', '경상북도', '인천광역시', '대전광역시',
       '강원도', '충청남도', '충청북도', '세종특별자치시', '광주광역시', '부산광역시', '제주특별자치도',
       '울산광역시', '전라북도'], dtype=object)

In [22]:
# 주소1, 주소2, 주소3 을 합쳐서 주소를 다시 입력
dat['주소'] = dat[ ['주소1', '주소2', '주소3'] ].apply(' '.join, axis=1)
dat.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5


In [23]:
# 수집된 데이터 전처리 전 저장
dat.to_excel('lotteria_all_store_v2.xlsx', index=False)
dat.to_csv('lotteria_all_store_v2.csv', index=False)
dat.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5


In [88]:
# 수집된 데이터 가져오기
dat = pd.read_excel('./lotteria_all_store_v2.xlsx')
dat

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5
5,신정네거리역,서울특별시 양천구 신월로 297,서울특별시,양천구,신월로 297
6,대구성당D/T,대구광역시 달서구 구마로 209,대구광역시,달서구,구마로 209
7,진주혁신,경상남도 진주시 소호로 12,경상남도,진주시,소호로 12
8,영암대불,전라남도 영암군 삼호읍 대불주거로 145,전라남도,영암군,삼호읍 대불주거로 145
9,안산사동,경기도 안산시 상록구 항가울로 28,경기도,안산시,상록구 항가울로 28


## 이상치, 결측치 처리

In [89]:
dat["주소2"].isna().sum()

6

### 1. nan 값 처리
- 세종특별자치시는 행정구가 없기 때문에 주소2에 세종시 입력
- 세종특별자치시는 1읍 9면 10행정동 18법정동으로 구성

In [90]:
dat[dat["주소1"] == "세종특별자치시"]

,지점명,주소,주소1,주소2,주소3
37,세종반곡,세종특별자치시 한누리대로 1824,세종특별자치시,NaN,한누리대로 1824
79,세종고운,세종특별자치시 마음안1로 137,세종특별자치시,NaN,마음안1로 137
80,세종보람디엠시티,세종특별자치시 한누리대로 2165,세종특별자치시,NaN,한누리대로 2165
83,세종소담,세종특별자치시 한누리대로 2009,세종특별자치시,NaN,한누리대로 2009
289,세종종촌,세종특별자치시 도움1로 108,세종특별자치시,NaN,도움1로 108
313,세종부강,세종특별자치시 부강면 부강로 42,세종특별자치시,NaN,부강면 부강로 42
373,세종첫마을,세종특별자치시 연기면 나성길 세종특별자치시 나성동 540번지,세종특별자치시,연기면,나성길 세종특별자치시 나성동 540번지
375,홈플러스세종,세종특별자치시 금남면 세종로 세종특별자치시 어진동 307-3,세종특별자치시,금남면,세종로 세종특별자치시 어진동 307-3
842,홈플러스조치원,세종특별자치시 조치원읍 허만석로 60-1,세종특별자치시,조치원읍,허만석로 60-1
891,조치원,세종특별자치시 조치원읍 조치원로 27-1,세종특별자치시,조치원읍,조치원로 27-1


In [91]:
dat.loc[dat["주소1"] == '세종특별자치시', '주소2'] = '세종특별자치시'
dat[dat["주소1"] == "세종특별자치시"]

,지점명,주소,주소1,주소2,주소3
37,세종반곡,세종특별자치시 한누리대로 1824,세종특별자치시,세종특별자치시,한누리대로 1824
79,세종고운,세종특별자치시 마음안1로 137,세종특별자치시,세종특별자치시,마음안1로 137
80,세종보람디엠시티,세종특별자치시 한누리대로 2165,세종특별자치시,세종특별자치시,한누리대로 2165
83,세종소담,세종특별자치시 한누리대로 2009,세종특별자치시,세종특별자치시,한누리대로 2009
289,세종종촌,세종특별자치시 도움1로 108,세종특별자치시,세종특별자치시,도움1로 108
313,세종부강,세종특별자치시 부강면 부강로 42,세종특별자치시,세종특별자치시,부강면 부강로 42
373,세종첫마을,세종특별자치시 연기면 나성길 세종특별자치시 나성동 540번지,세종특별자치시,세종특별자치시,나성길 세종특별자치시 나성동 540번지
375,홈플러스세종,세종특별자치시 금남면 세종로 세종특별자치시 어진동 307-3,세종특별자치시,세종특별자치시,세종로 세종특별자치시 어진동 307-3
842,홈플러스조치원,세종특별자치시 조치원읍 허만석로 60-1,세종특별자치시,세종특별자치시,허만석로 60-1
891,조치원,세종특별자치시 조치원읍 조치원로 27-1,세종특별자치시,세종특별자치시,조치원로 27-1


In [92]:
dat.loc[dat["지점명"] == '세종첫마을', '주소3'] = '나성동 540번지'
dat.loc[dat["지점명"] == '홈플러스세종', '주소3'] = '어진동 307-3'
dat[dat["주소1"] == "세종특별자치시"]

,지점명,주소,주소1,주소2,주소3
37,세종반곡,세종특별자치시 한누리대로 1824,세종특별자치시,세종특별자치시,한누리대로 1824
79,세종고운,세종특별자치시 마음안1로 137,세종특별자치시,세종특별자치시,마음안1로 137
80,세종보람디엠시티,세종특별자치시 한누리대로 2165,세종특별자치시,세종특별자치시,한누리대로 2165
83,세종소담,세종특별자치시 한누리대로 2009,세종특별자치시,세종특별자치시,한누리대로 2009
289,세종종촌,세종특별자치시 도움1로 108,세종특별자치시,세종특별자치시,도움1로 108
313,세종부강,세종특별자치시 부강면 부강로 42,세종특별자치시,세종특별자치시,부강면 부강로 42
373,세종첫마을,세종특별자치시 연기면 나성길 세종특별자치시 나성동 540번지,세종특별자치시,세종특별자치시,나성동 540번지
375,홈플러스세종,세종특별자치시 금남면 세종로 세종특별자치시 어진동 307-3,세종특별자치시,세종특별자치시,어진동 307-3
842,홈플러스조치원,세종특별자치시 조치원읍 허만석로 60-1,세종특별자치시,세종특별자치시,허만석로 60-1
891,조치원,세종특별자치시 조치원읍 조치원로 27-1,세종특별자치시,세종특별자치시,조치원로 27-1


### 2. 주소2 데이터 값이 이상한 경우
1. 군+읍/면가 합쳐서 나온 경우 (예) 나주시금천면
    - 분리시켜주기
2. 시,군,구,읍,면 등과 같은 행정구역 단위로 끝나지 않는 경우 (예) 구리, 종로2가
    - 값을 직접 지정해주기

In [93]:
dat["주소2"].unique()

array(['가평군', '용산구', '수원시', '여수시', '김포시', '양천구', '달서구', '진주시', '영암군',
       '안산시', '의정부시', '용인시', '김해시', '포항시', '연수구', '광명시', '문경시', '서초구',
       '서구', '원주시', '무안군', '시흥시', '강동구', '서천군', '청주시', '남양주시', '화성시',
       '하남시', '동구', '마포구', '고양시', '논산시', '세종특별자치시', '천안시', '남구', '사하구',
       '구미시', '북구', '충주시', '안양시', '광주시', '부평구', '평택시', '양양군', '제주시',
       '홍천군', '전주시', '이천시', '은평구', '강서구', '거제시', '산청군', '군포시', '서대문구',
       '달성군', '중구', '수성구', '창원시', '진도군', '남동구', '평창군', '당진시', '성남시',
       '의령군', '익산시', '제천시', '동래구', '부천시', '광산구', '진안군', '안성시', '경산시',
       '노원구', '서산시', '서귀포시', '파주시', '밀양시', '성동구', '금정구', '순천시', '안동시',
       '의성군', '곡성군', '해운대구', '강남구', '부산진구', '영주시', '음성군', '춘천시', '금천구',
       '수영구', '구리시', '관악구', '동대문구', '장수군', '경주시', '보성군', '울릉군', '속초시',
       '인제군', '양주시', '울주군', '송파구', '영등포구', '유성구', '옹진군', '아산시', '오산시',
       '양산시', '중랑구', '정읍시', '의왕시', '동작구', '고성군', '함평군', '김천시', '홍성군',
       '보령시', '무주군', '구례군', '청원군', '강북구', '당진군', '구로구', '강릉시', '여주군',
       '기장군',

In [94]:
# 주소2의 이름이 시,군,구,동으로 끝나지 않거나 길이가 긴 경우 exc에 저장

exc = []
for one in dat["주소2"].unique():
    try :
        if len(one)>5:
            if one == '세종특별자치시' :
                pass
            else :
                exc.append(one)
        elif one[-1] not in ['시','구','군','동','면',"읍"] :
            exc.append(one)
    except :
        print(one)

exc

['나주시금천면',
 '장성군장성읍',
 '고흥군고흥읍',
 '논산시연무읍',
 '임실군임실읍',
 '창녕군남지읍',
 '아산신',
 '종로2가',
 '구로',
 '남양주시화도읍']

In [95]:
si = ['시','구','군']
idx_list = []

In [96]:
for i in exc :
    
    idx_list.append(dat.loc[dat['주소2'] == i].index[0])
    
    if len(i)>5:
        idx = 0
        
        for j in si :
            try :
                idx = int(i.index(j))
            except :
                pass
        
        addr3 = i[idx+1:] + ' ' + dat.loc[dat['주소2'] == i, '주소3'].values[0]

        dat.loc[dat['주소2'] == i, '주소3'] = addr3
        dat.loc[dat['주소2'] == i, '주소2'] = i[:idx+1]
        
    elif i == "구로":
        dat.loc[dat['주소2'] == i, '주소2'] = '구로구'
    elif i == "종로2가":
        dat.loc[dat['주소2'] == i, '주소2'] = '종로구'
    elif i == "아산신":
        dat.loc[dat['주소2'] == i, '주소2'] = '아산시'

In [97]:
dat.iloc[idx_list]

,지점명,주소,주소1,주소2,주소3
366,나주혁신,전라남도 나주시금천면 빛가람로 189-1,전라남도,나주시,금천면 빛가람로 189-1
368,전남장성,전라남도 장성군장성읍 영천로 166,전라남도,장성군,장성읍 영천로 166
404,전남고흥,전라남도 고흥군고흥읍 여산당촌길 8,전라남도,고흥군,고흥읍 여산당촌길 8
418,논산연무,충청남도 논산시연무읍 안심로 152,충청남도,논산시,연무읍 안심로 152
427,임실,전라북도 임실군임실읍 운수로 19,전라북도,임실군,임실읍 운수로 19
434,경남남지,경상남도 창녕군남지읍 문화길 35-4,경상남도,창녕군,남지읍 문화길 35-4
678,아산탕정,충청남도 아산신 탕정면 탕정면로 37,충청남도,아산시,탕정면 탕정면로 37
984,종각역,서울특별시 종로2가 종로 68,서울특별시,종로구,종로 68
1040,개봉역,서울특별시 구로 경인로 40길 47 1F,서울특별시,구로구,경인로 40길 47 1F
1140,마석,경기도 남양주시화도읍 마석로 14,경기도,남양주시,화도읍 마석로 14


### 확인

In [98]:
addr = dat["주소2"].unique()
print(addr)

result = True
for one in addr:
    if type(one)==float :
        result = False
        print(one)
    elif one[-1] not in ["시","구","군","동","면","읍"]:
        result = False
        print(one)
    elif len(one)>5:
        if one == '세종특별자치시' :
            pass
        else :
            result =False
        print(one)

if result :
    print("결과 : 이상치가 없습니다.")
else :
    print("결과 : 이상치를 모두 출력하였습니다.")

['가평군' '용산구' '수원시' '여수시' '김포시' '양천구' '달서구' '진주시' '영암군' '안산시' '의정부시' '용인시'
 '김해시' '포항시' '연수구' '광명시' '문경시' '서초구' '서구' '원주시' '무안군' '시흥시' '강동구' '서천군'
 '청주시' '남양주시' '화성시' '하남시' '동구' '마포구' '고양시' '논산시' '세종특별자치시' '천안시' '남구'
 '사하구' '구미시' '북구' '충주시' '안양시' '광주시' '부평구' '평택시' '양양군' '제주시' '홍천군' '전주시'
 '이천시' '은평구' '강서구' '거제시' '산청군' '군포시' '서대문구' '달성군' '중구' '수성구' '창원시' '진도군'
 '남동구' '평창군' '당진시' '성남시' '의령군' '익산시' '제천시' '동래구' '부천시' '광산구' '진안군' '안성시'
 '경산시' '노원구' '서산시' '서귀포시' '파주시' '밀양시' '성동구' '금정구' '순천시' '안동시' '의성군' '곡성군'
 '해운대구' '강남구' '부산진구' '영주시' '음성군' '춘천시' '금천구' '수영구' '구리시' '관악구' '동대문구'
 '장수군' '경주시' '보성군' '울릉군' '속초시' '인제군' '양주시' '울주군' '송파구' '영등포구' '유성구' '옹진군'
 '아산시' '오산시' '양산시' '중랑구' '정읍시' '의왕시' '동작구' '고성군' '함평군' '김천시' '홍성군' '보령시'
 '무주군' '구례군' '청원군' '강북구' '당진군' '구로구' '강릉시' '여주군' '기장군' '나주시' '장성군' '사상구'
 '봉화군' '하동군' '성북구' '양평군' '고령군' '연제구' '고흥군' '순창군' '임실군' '창녕군' '광진구' '영천시'
 '완주군' '담양군' '함안군' '부안군' '성주군' '화천군' '강진군' '부여군' '장흥군' '군산시' '대덕구' '도봉구'
 '계양구' '여주시' '동두천시' '목포시' '진구' '남원시' '정선군' '종로구' '동해시' '

### 3. 잘못된 정보 처리
- 천안은 충청남도인데 충청북도라고 나와있는 데이터가 있다.

In [99]:
temp_idx = dat[(dat["주소1"]=="충청북도")&(dat["주소2"]=="천안시")].index[0]
dat[(dat["주소1"]=="충청북도")&(dat["주소2"]=="천안시")]

,지점명,주소,주소1,주소2,주소3
506,천안안서D/T,충청북도 천안시 동남구 망향로 190,충청북도,천안시,동남구 망향로 190


In [100]:
dat.iloc[temp_idx,1] = "충청남도 천안시 동남구 망향로 190"
dat.iloc[temp_idx,2] = "충청남도"

dat.iloc[temp_idx]

지점명                 천안안서D/T
주소     충청남도 천안시 동남구 망향로 190
주소1                    충청남도
주소2                     천안시
주소3             동남구 망향로 190
Name: 506, dtype: object

In [102]:
# 전처리 된 데이터 저장
dat.to_excel('lotteria_all_store_v3.xlsx', index=False)
dat.to_csv('lotteria_all_store_v3.csv', index=False)
dat

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5
5,신정네거리역,서울특별시 양천구 신월로 297,서울특별시,양천구,신월로 297
6,대구성당D/T,대구광역시 달서구 구마로 209,대구광역시,달서구,구마로 209
7,진주혁신,경상남도 진주시 소호로 12,경상남도,진주시,소호로 12
8,영암대불,전라남도 영암군 삼호읍 대불주거로 145,전라남도,영암군,삼호읍 대불주거로 145
9,안산사동,경기도 안산시 상록구 항가울로 28,경기도,안산시,상록구 항가울로 28


In [190]:
# 수집된 데이터 가져오기
dat = pd.read_excel('./lotteria_all_store_v3.xlsx')
dat

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5
5,신정네거리역,서울특별시 양천구 신월로 297,서울특별시,양천구,신월로 297
6,대구성당D/T,대구광역시 달서구 구마로 209,대구광역시,달서구,구마로 209
7,진주혁신,경상남도 진주시 소호로 12,경상남도,진주시,소호로 12
8,영암대불,전라남도 영암군 삼호읍 대불주거로 145,전라남도,영암군,삼호읍 대불주거로 145
9,안산사동,경기도 안산시 상록구 항가울로 28,경기도,안산시,상록구 항가울로 28


2021.07.07 12:00 김민수 update
### - 데이터 1개 : ‘경기도 여주군’ → ‘경기도 여주시’로 변경 
  - 여주가남점

In [191]:
dat.loc[ dat['주소2'] == '여주군']

,지점명,주소,주소1,주소2,주소3
356,여주가남,경기도 여주군 가남면 태평리 110-7,경기도,여주군,가남면 태평리 110-7


In [192]:
dat.loc[ dat['주소2'] == '여주군', '주소2'] = '여주시'

In [193]:
dat.loc[ dat['주소2'] == '여주시']

,지점명,주소,주소1,주소2,주소3
356,여주가남,경기도 여주군 가남면 태평리 110-7,경기도,여주시,가남면 태평리 110-7
543,여주375아울렛,경기도 여주시 명품1로 22-5,경기도,여주시,명품1로 22-5
1169,여주,경기도 여주시 세종로 21,경기도,여주시,세종로 21


### - ‘인천광역시 남구’ → ‘인천광역시 미추홀구‘로 변경
  - 동일한 지역인데 KFC, 버거킹, 맥도날드는 ‘인천광역시 미추홀구‘로 되어있음
  - 시군구별 인구수,인구밀도 데이터에 ‘인천광역시 미추홀구‘로 되어 있어서 이것으로 통일하면 좋을 듯

In [194]:
dat.loc[ (dat['주소1'] == '인천광역시') & (dat['주소2'] == '남구') ]

,지점명,주소,주소1,주소2,주소3
117,롯데백인천터미널,인천광역시 남구 연남로 35,인천광역시,남구,연남로 35
142,인천도화,인천광역시 남구 숙골로 100,인천광역시,남구,숙골로 100
298,주안,인천광역시 남구 주안로 지하86,인천광역시,남구,주안로 지하86
310,구월아시아드,인천광역시 남구 인하로 563,인천광역시,남구,인하로 563
315,인천학산,인천광역시 남구 한나루로 389,인천광역시,남구,한나루로 389
557,홈플러스인천숭의,인천광역시 남구 석정로 51,인천광역시,남구,석정로 51
569,인천석바위,인천광역시 남구 경원대로848,인천광역시,남구,경원대로848
776,인천간석,인천광역시 남구 주안로219,인천광역시,남구,주안로219
917,홈플러스인하,인천광역시 남구 소성로 6,인천광역시,남구,소성로 6
1053,인천용현,인천광역시 남구 낙섬중로 42,인천광역시,남구,낙섬중로 42


In [195]:
dat.loc[ (dat['주소1'] == '인천광역시') & (dat['주소2'] == '남구'), '주소2' ] = '미추홀구'

In [196]:
dat.loc[ (dat['주소1'] == '인천광역시') & (dat['주소2'] == '미추홀구') ]

,지점명,주소,주소1,주소2,주소3
117,롯데백인천터미널,인천광역시 남구 연남로 35,인천광역시,미추홀구,연남로 35
142,인천도화,인천광역시 남구 숙골로 100,인천광역시,미추홀구,숙골로 100
298,주안,인천광역시 남구 주안로 지하86,인천광역시,미추홀구,주안로 지하86
310,구월아시아드,인천광역시 남구 인하로 563,인천광역시,미추홀구,인하로 563
315,인천학산,인천광역시 남구 한나루로 389,인천광역시,미추홀구,한나루로 389
557,홈플러스인천숭의,인천광역시 남구 석정로 51,인천광역시,미추홀구,석정로 51
569,인천석바위,인천광역시 남구 경원대로848,인천광역시,미추홀구,경원대로848
776,인천간석,인천광역시 남구 주안로219,인천광역시,미추홀구,주안로219
917,홈플러스인하,인천광역시 남구 소성로 6,인천광역시,미추홀구,소성로 6
1053,인천용현,인천광역시 남구 낙섬중로 42,인천광역시,미추홀구,낙섬중로 42


### - 주소에 “광주광역시 광역시“라고 되어있는 데이터 3개 있음 
  - 광주금남로, 광주동림, 광주오치

In [197]:
idx_list2 = list(dat.loc[ dat['주소2'] == '광역시' ].index)
dat.loc[ dat['주소2'] == '광역시' ]

,지점명,주소,주소1,주소2,주소3
774,광주동림,광주광역시 광역시 서구 동천동626,광주광역시,광역시,서구 동천동626
1081,광주오치,광주광역시 광역시 북구 오치동 949-27,광주광역시,광역시,북구 오치동 949-27
1285,광주금남로,광주광역시 광역시 동구 서석로51,광주광역시,광역시,동구 서석로51


In [198]:
def change_addr2(x) :
    idx = x['주소3'].index('구') + 1
    
    addr2 = x['주소3'][:idx]
    addr3 = x['주소3'][idx+1:]
    
    x['주소2', '주소3'] = addr2, addr3
    
    return x
    
dat = dat.apply(lambda x : change_addr2(x) if x['주소2'] == '광역시' else x, axis=1)

In [199]:
dat.iloc[ idx_list2 ]

,지점명,주소,주소1,주소2,주소3
774,광주동림,광주광역시 광역시 서구 동천동626,광주광역시,서구,동천동626
1081,광주오치,광주광역시 광역시 북구 오치동 949-27,광주광역시,북구,오치동 949-27
1285,광주금남로,광주광역시 광역시 동구 서석로51,광주광역시,동구,서석로51


In [200]:
# 주소1, 주소2, 주소3 을 합쳐서 주소를 다시 입력
dat['주소'] = dat[ ['주소1', '주소2', '주소3'] ].apply(' '.join, axis=1)
dat.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5


In [201]:
dat.loc[dat["주소1"] == "세종특별자치시"]

,지점명,주소,주소1,주소2,주소3
37,세종반곡,세종특별자치시 세종특별자치시 한누리대로 1824,세종특별자치시,세종특별자치시,한누리대로 1824
79,세종고운,세종특별자치시 세종특별자치시 마음안1로 137,세종특별자치시,세종특별자치시,마음안1로 137
80,세종보람디엠시티,세종특별자치시 세종특별자치시 한누리대로 2165,세종특별자치시,세종특별자치시,한누리대로 2165
83,세종소담,세종특별자치시 세종특별자치시 한누리대로 2009,세종특별자치시,세종특별자치시,한누리대로 2009
289,세종종촌,세종특별자치시 세종특별자치시 도움1로 108,세종특별자치시,세종특별자치시,도움1로 108
313,세종부강,세종특별자치시 세종특별자치시 부강면 부강로 42,세종특별자치시,세종특별자치시,부강면 부강로 42
373,세종첫마을,세종특별자치시 세종특별자치시 나성동 540번지,세종특별자치시,세종특별자치시,나성동 540번지
375,홈플러스세종,세종특별자치시 세종특별자치시 어진동 307-3,세종특별자치시,세종특별자치시,어진동 307-3
842,홈플러스조치원,세종특별자치시 세종특별자치시 허만석로 60-1,세종특별자치시,세종특별자치시,허만석로 60-1
891,조치원,세종특별자치시 세종특별자치시 조치원로 27-1,세종특별자치시,세종특별자치시,조치원로 27-1


In [202]:
dat['주소'] = dat.apply(lambda x : x['주소1'] + ' ' + x['주소3'] if x['주소1'] == '세종특별자치시' else x['주소'], axis=1)

In [203]:
dat.loc[dat["주소1"] == "세종특별자치시"]

,지점명,주소,주소1,주소2,주소3
37,세종반곡,세종특별자치시 한누리대로 1824,세종특별자치시,세종특별자치시,한누리대로 1824
79,세종고운,세종특별자치시 마음안1로 137,세종특별자치시,세종특별자치시,마음안1로 137
80,세종보람디엠시티,세종특별자치시 한누리대로 2165,세종특별자치시,세종특별자치시,한누리대로 2165
83,세종소담,세종특별자치시 한누리대로 2009,세종특별자치시,세종특별자치시,한누리대로 2009
289,세종종촌,세종특별자치시 도움1로 108,세종특별자치시,세종특별자치시,도움1로 108
313,세종부강,세종특별자치시 부강면 부강로 42,세종특별자치시,세종특별자치시,부강면 부강로 42
373,세종첫마을,세종특별자치시 나성동 540번지,세종특별자치시,세종특별자치시,나성동 540번지
375,홈플러스세종,세종특별자치시 어진동 307-3,세종특별자치시,세종특별자치시,어진동 307-3
842,홈플러스조치원,세종특별자치시 허만석로 60-1,세종특별자치시,세종특별자치시,허만석로 60-1
891,조치원,세종특별자치시 조치원로 27-1,세종특별자치시,세종특별자치시,조치원로 27-1


In [204]:
# 전처리 된 데이터 저장
dat.to_excel('lotteria_all_store.xlsx', index=False)
dat.to_csv('lotteria_all_store.csv', index=False)
dat

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5
5,신정네거리역,서울특별시 양천구 신월로 297,서울특별시,양천구,신월로 297
6,대구성당D/T,대구광역시 달서구 구마로 209,대구광역시,달서구,구마로 209
7,진주혁신,경상남도 진주시 소호로 12,경상남도,진주시,소호로 12
8,영암대불,전라남도 영암군 삼호읍 대불주거로 145,전라남도,영암군,삼호읍 대불주거로 145
9,안산사동,경기도 안산시 상록구 항가울로 28,경기도,안산시,상록구 항가울로 28


In [205]:
import os
os.getcwd()

'C:\\Users\\Su\\Documents\\GitHub\\burger-index\\1. web_crawling+preprocessing\\Lotteria'

In [206]:
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Lotteria.ipynb',
 'lotteria_all_store.csv',
 'lotteria_all_store.xlsx',
 'lotteria_all_store_v1.csv',
 'lotteria_all_store_v1.xlsx',
 'lotteria_all_store_v2.csv',
 'lotteria_all_store_v2.xlsx',
 'lotteria_all_store_v3.csv',
 'lotteria_all_store_v3.xlsx']

2021.07.07 14:30 김동화 update

In [3]:
# 데이터 불러오기, 기존 데이터 v4으로 저장
dat = pd.read_csv("lotteria_all_store.csv")
dat.to_excel('lotteria_all_store_v4.xlsx', index=False)
dat.to_csv('lotteria_all_store_v4.csv', index=False)
dat

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5
...,...,...,...,...,...
1299,원주일산,강원도 원주시 천사로 213,강원도,원주시,천사로 213
1300,성남,경기도 성남시 수정구 수정로 181,경기도,성남시,수정구 수정로 181
1301,대전중앙,대전광역시 중구 선화동 3-3번지,대전광역시,중구,선화동 3-3번지
1302,홈서비스과천,경기도 과천시 별양동 19-4,경기도,과천시,별양동 19-4


### '충청북도 청원군' 처리 : 청원군은 청주시 소속 

In [10]:
idx_cw = dat.loc[dat["주소2"]=='청원군','주소2'].index
dat.loc[dat['주소2']=='청원군','주소2'] = '청주시'
dat.iloc[idx_cw]

,지점명,주소,주소1,주소2,주소3
324,청주강내,충청북도 청원군 강내면 탑연리 277-2,충청북도,청주시,강내면 탑연리 277-2
694,충북오송,충청북도 청원군 오송읍 오송생명로 194,충청북도,청주시,오송읍 오송생명로 194
832,홈플러스오창,충청북도 청원군 오창읍 중심상업1로 8-11,충청북도,청주시,오창읍 중심상업1로 8-11
900,오창,충청북도 청원군 오창읍 중심상업로 7,충청북도,청주시,오창읍 중심상업로 7


### '충청남도 당진군' 처리 : '충청남도 당진시'으로

In [13]:
idx_dj = dat[dat["주소2"]=='당진군'].index
dat.loc[dat['주소2']=='당진군','주소2'] = '당진시'
dat.iloc[idx_dj]

,지점명,주소,주소1,주소2,주소3
333,당진신평,충청남도 당진군 신평면 거산리 15-6,충청남도,당진시,신평면 거산리 15-6


### '울산광역시 을주군' 처리 : '울산광역시 울주군'으로

In [17]:
idx_wj = dat[dat["주소2"]=='을주군'].index
dat.loc[dat['주소2']=='을주군','주소2'] = '울주군'
dat.iloc[idx_wj]

,지점명,주소,주소1,주소2,주소3
759,울산온양,울산광역시 을주군 온양읍 온양로 95,울산광역시,울주군,온양읍 온양로 95


### '부산 진구' 처리 : '부산 진구'로 통일

In [21]:
idx_bs = dat[dat['주소2']=='진구'].index
dat.loc[dat['주소2']=="진구","주소2"] = "부산진구"
dat.iloc[idx_bs]

,지점명,주소,주소1,주소2,주소3
560,홈플러스부산가야,부산광역시 진구 가야대로 506,부산광역시,부산진구,가야대로 506
685,키즈마트부산,부산광역시 진구 신천대로 241,부산광역시,부산진구,신천대로 241


In [22]:
dat.to_excel('lotteria_all_store.xlsx', index=False)
dat.to_csv('lotteria_all_store.csv', index=False)
dat

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5
...,...,...,...,...,...
1299,원주일산,강원도 원주시 천사로 213,강원도,원주시,천사로 213
1300,성남,경기도 성남시 수정구 수정로 181,경기도,성남시,수정구 수정로 181
1301,대전중앙,대전광역시 중구 선화동 3-3번지,대전광역시,중구,선화동 3-3번지
1302,홈서비스과천,경기도 과천시 별양동 19-4,경기도,과천시,별양동 19-4
